In [119]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from src.logit_gb import LogitGB

In [6]:
with open('models/model_up.p', 'rb') as file:
    model_up = pickle.load(file)
with open('locations.p', 'rb') as file:
    locations = pickle.load(file)
with open('models/vec.p', 'rb') as file:
    vec = pickle.load(file)
with open('data/predictor.p', 'rb') as file:
    predictor = pickle.load(file)
with open('data/text.p', 'rb') as file:
    text = pickle.load(file)

In [40]:
with open('train/fires_train.p', 'rb') as file:
    target_train = pickle.load(file)

In [11]:
with open('train/target_matching_train.p', 'rb') as file:
    train = pickle.load(file)

In [7]:
predictor['has_existing_stories'] = (predictor['existing_stories'] > 0).astype(int)

In [81]:
ls train

 Volume in drive C is Windows
 Volume Serial Number is F25C-FBB9

 Directory of C:\Users\pengj\OneDrive\Documents\School\galvanize\sf-fires\train

04/03/2019  08:43 AM    <DIR>          .
04/03/2019  08:43 AM    <DIR>          ..
04/02/2019  08:51 PM         6,769,747 fires.p
04/02/2019  05:27 PM         2,580,249 fires_df.p
04/02/2019  05:36 PM         2,847,327 fires_df_train.p
04/03/2019  08:43 AM         5,622,995 fires_train.p
04/02/2019  05:34 PM            38,161 target_date_index.p
04/02/2019  05:28 PM        15,557,551 target_matching.p
04/02/2019  05:37 PM        15,734,575 target_matching_train.p
               7 File(s)     49,150,605 bytes
               2 Dir(s)  11,163,066,368 bytes free


In [82]:
with open('train/fires.p', 'rb') as file:
    fires = pickle.load(file)

In [96]:
predictor['fires_before'] = fires[:,0]
predictor['fires_after'] = fires[:,1]
predictor['fire_rate_before'] = 365*predictor['fires_before']/(predictor['status_date'] - pd.to_datetime('2003-01-01')).apply(lambda x: x.days)
predictor['fire_rate_after'] = 365*predictor['fires_before']/(-(predictor['status_date'] - pd.to_datetime('2017-12-31'))).apply(lambda x: x.days)

In [98]:
predictor['existing_stories'] = predictor['existing_stories'].fillna(0)

In [99]:
with open('data/full_df.p', 'wb') as file:
    pickle.dump(predictor, file)

In [126]:
fires

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [1., 0.]])

In [102]:
df_test = predictor.iloc[test_mask]

In [109]:
X_text_test = vec.transform(text[test_mask])

In [91]:
test_mask

array([False, False, False, ...,  True,  True,  True])

In [17]:
predictor_train = predictor.iloc[train.index]

In [18]:
n = 30000
sample = np.random.choice(predictor_train.shape[0], n)

In [42]:
target_train.shape

(351427, 2)

In [29]:
text_df_train = text[train.index][sample]

In [31]:
text_mat = vec.transform(text_df_train)

In [79]:
numerical_df = predictor_train.iloc[sample].fillna(0)

model = LogitGB()

In [73]:
target_train = predictor_train[['status_date', 'fires_after', 'fire_rate_after', 'fires_before', 'fire_rate_before']]

In [68]:
with open('data/full_train_data.p', 'wb') as file:
    pickle.dump(predictor_train, file)

In [80]:
model.fit(text_mat, numerical_df, target_train.iloc[sample])

C:\Users\pengj\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


(71672,)

In [114]:
y_test = (df_test['fire_rate_after'] > df_test['fire_rate_before']).astype(int)

In [122]:
accuracy_score(y_test, model.predict(df_test, X_text_test))

0.9128250920861704

In [125]:
1 - y_test.mean()

1.0